In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)


In [2]:
querys={
    
    'Vendas':
    
    """
    
    SELECT ped.[Data de Emissão],cli.[Data de Cadastro],
    CASE WHEN ped.[Data de Emissão]=cli.[Data de Cadastro] THEN 'S' ELSE 'N' END AS [Positivado],
    ped.Pedido,ped.[ID Cliente],cli.[Razão Social],cli.[Nome Fantasia],
    ped.[ID Vendedor],vend.[Nome Resumido],vend.DDD,vend.Telefone,sup.Equipe,
    sup.[ID Sup],sup.Supervisor,sup.[DDD Sup],sup.[Telefone Sup],
    sup.[ID Gerente],sup.Gerente,sup.[DDD Gerente],sup.[Telefone Gerente],
    SUM(ped.[Total Venda]) AS [Total Venda]
    FROM netfeira.vw_venda_estatico ped
    INNER JOIN netfeira.vw_cliente cli ON ped.[ID Cliente]=cli.[ID Cliente]
    INNER JOIN netfeira.vw_vendedor vend ON ped.[ID Vendedor]=vend.[ID Vendedor] AND vend.Categoria='CLT'
    INNER JOIN netfeira.vw_supervisor sup ON vend.[ID Equipe]=sup.[ID Equipe]
    WHERE [Data de Emissão] BETWEEN DATEADD(DAY,1,
    DATEADD(DAY,DAY(DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)))*-1,
    DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)))) AND
    DATEADD(DAY,DAY(GETDATE())*-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101))
    AND [Tipo de Operação]='VENDAS'
    GROUP BY ped.[Data de Emissão],cli.[Data de Cadastro],ped.Pedido,ped.[ID Cliente],cli.[Razão Social],cli.[Nome Fantasia],
    ped.[ID Vendedor],vend.[Nome Resumido],vend.DDD,vend.Telefone,sup.Equipe,
    sup.[ID Sup],sup.Supervisor,sup.[DDD Sup],sup.[Telefone Sup],
    sup.[ID Gerente],sup.Gerente,sup.[DDD Gerente],sup.[Telefone Gerente]    
    
    """
}

In [ ]:
df=sql.CriarTabela(kwargs=querys)

In [ ]:
df['Lista']=df['Vendas'].groupby(['ID Cliente'],as_index=False).agg({'Total Venda':'sum'})

In [ ]:
codigos=df['Lista'].loc[df['Lista']['Total Venda']>0,'ID Cliente'].unique().tolist()

In [ ]:
df['Vendas'].columns

In [ ]:
df['Consolidado']=df['Vendas'].loc[(df['Vendas']['ID Cliente'].isin(codigos))&(df['Vendas']['Positivado']=='S')].groupby(['ID Cliente', 'Razão Social', 'Nome Fantasia','Data de Cadastro', 'ID Vendedor',
       'Nome Resumido', 'DDD', 'Telefone', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],as_index=False).agg({'Total Venda':'sum'})

In [ ]:
df['Consolidado']['Data Mín']=df['Consolidado']['ID Cliente'].apply(lambda info: df['Vendas'].loc[df['Vendas']['ID Cliente']==info,'Data de Emissão'].min())

In [ ]:
df['Consolidado']['Pedido']=df['Consolidado']['ID Cliente'].apply(lambda info: df['Vendas'].loc[df['Vendas']['ID Cliente']==info,'Pedido'].min())

In [ ]:
df['Temp']=df['Consolidado'].groupby(['ID Vendedor',
       'Nome Resumido', 'DDD', 'Telefone', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],as_index=False).agg({'ID Cliente':'count'}).sort_values('ID Cliente',ascending=False)

In [ ]:
df['Temp']